# Preprocessing the waitrose data

__Author:__ Casey Kearney

__Last Updated:__ 2024/10/26

# 1. Read in the Data

In [2]:
import pandas as pd
import numpy as np
import os

In [5]:
# Create folder path to our data
data_folder = os.path.join('..', 'data', 'waitrose-2024-07')

In [ ]:
# Goal is to read all the files in this folder
for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        print(os.path.join(data_folder, file)) 

In [13]:
# Create a list of all the files:
all_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.csv')]

In [15]:
# Have a list of all the file names
print(all_files)
len(all_files)

['..\\data\\waitrose-2024-07\\baby-child-and-parent.csv', '..\\data\\waitrose-2024-07\\bakery.csv', '..\\data\\waitrose-2024-07\\beer-wine-and-spirits.csv', '..\\data\\waitrose-2024-07\\best-of-british.csv', '..\\data\\waitrose-2024-07\\dietary-and-lifestyle.csv', '..\\data\\waitrose-2024-07\\everyday-value.csv', '..\\data\\waitrose-2024-07\\food-cupboard.csv', '..\\data\\waitrose-2024-07\\fresh-and-chilled.csv', '..\\data\\waitrose-2024-07\\frozen.csv', '..\\data\\waitrose-2024-07\\home.csv', '..\\data\\waitrose-2024-07\\household.csv', '..\\data\\waitrose-2024-07\\new.csv', '..\\data\\waitrose-2024-07\\organic-shop.csv', '..\\data\\waitrose-2024-07\\pet.csv', '..\\data\\waitrose-2024-07\\summer.csv', '..\\data\\waitrose-2024-07\\tea-coffee-and-soft-drinks.csv', '..\\data\\waitrose-2024-07\\toiletries-health-and-beauty.csv', '..\\data\\waitrose-2024-07\\waitrose-brands.csv']


18

In [16]:
# Want to loop over this list to save data
# For now, create a list of data frames
list_of_dfs = [pd.read_csv(file) for file in all_files]


In [ ]:
# first element is a dataframe
list_of_dfs[0]

In [23]:
# Total rows
rows = 0
for df in list_of_dfs:
    rows += df.shape[0]

rows

25418

In [110]:
# We can then merge all of these files at once
df = pd.concat(list_of_dfs)

In [111]:
# Dimensions align with what we want
df.shape

(25418, 13)

# Puzzle 1 - Explore the `item-price` column

In [112]:
df['item-price'].describe()

count     25407
unique      774
top       £2.50
freq        948
Name: item-price, dtype: object

In [113]:
# Need to eliminate currency notation
df['item-price'].head()

0     £3.15
1     £3.15
2    £13.50
3       95p
4     £1.60
Name: item-price, dtype: object

In [32]:
# Strip leading currency symbol
df['item-price'] = df['item-price'].str.replace('£', '')

# TODO Confirm all values ending in p are less than £1


# TODO Convert values ending in p to appropriate value



In [35]:
# Count number ending in p
df['item-price'].str.endswith('p').sum()

734

In [70]:
# Check for missing values in the 
pence_item_rows = (df['item-price'].str.endswith('p') == True)

In [75]:
# Every value is 3 characters, with the p
df['item-price'][pence_item_rows].str.len()

3       3
27      3
29      3
39      3
43      3
       ..
1281    3
1284    3
1352    3
1576    3
1577    3
Name: item-price, Length: 734, dtype: int64

In [78]:
# Write function to fix pence values
def get_pence_value(row):
    if row['item-price'].str.endswith('p'):
        # Replace the string and add a leading 0
        return '0.' + row['item-price'].str.replace('p', '')
    else:
        return row['item-price']

In [79]:
get_pence_value(df)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:
item_prices = [
    '0.' + item_price.replace('p', '') for item_price in df['item-price']
    if str(item_price).endswith('p')
]

In [89]:
df.head()

,data-product-id,data-product-name,data-product-type,data-product-on-offer,data-product-index,image-url,product-page,product-name,product-size,item-price,price-per-unit,offer-description,category
0,525635,Organix Raspberry & Apple Soft Oaty Bars,G,False,1.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
1,557746,Organix Carrot Cake Oaty Bars,G,False,2.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,Aptamil 2 Follow On Milk,G,False,394.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent"
3,767801,Essential Baby Wipes,G,False,4.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,95p,1.2p each,NaN,"Baby, Child & Parent"
4,514054,Organix Apple Rice Cakes,G,False,5.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent"


In [105]:
# Could we just loop over the rows
index = 0
for row in df['item-price']:
    if str(row).endswith('p'):
        df.loc[index, 'item-price'] = '0.' + str(row).replace('p','')
    index += 1

In [106]:
# Appears to have worked okay, although slight more than I would have liked
df['item-price'].str.startswith('0.').sum()

1297

In [107]:
index

25995

In [108]:
# Still have many instances of this
df['item-price'].str.endswith('p').sum()

697

In [109]:
df['item-price'].astype(float)

ValueError: could not convert string to float: '65p'

# 2. Removing the currency symbol

In [170]:
def clean_item_price(item_price: str):
    # Convert the input to a string and strip
    item_price = str(item_price).strip()
    
    # Replace the currency symbol
    item_price = item_price.replace('£', '')

    # Replace 'each est.'
    item_price = item_price.replace('each est.', '').strip()

    # Eliminate the pence and replace with '0.'
    if item_price.endswith('p'):
        item_price = '0.' + item_price.strip('p').strip()
    
    if item_price.endswith('each est.'):
        item_price = item_price.replace('each est.', '')

    if item_price.endswith('p'):
        item_price = '0.' + item_price.strip('p').strip()
    
    # Strip empty white space
    item_price = item_price.strip()
    
    # Return the cleaned string as a string
    return item_price

In [171]:
df['item-price'].apply(clean_item_price)

0        3.15
1        3.15
2       13.50
3        0.95
4        1.60
        ...  
1589    24.31
1590     2.40
1591    10.00
1592     2.50
1593    21.49
Name: item-price, Length: 25418, dtype: object

In [177]:
'-' in '55-£110 each est.'

True

In [176]:
'55-£110 each est.'.split('-')[0]

'55'

In [174]:
clean_item_price('£55-£110 each est.')

'55-110'

In [173]:
df['item-price'][
pd.to_numeric(df['item-price'].apply(clean_item_price), errors='coerce').isna()
]


291      £55-£110 each est.
294       £30-£38 each est.
299       £54-£92 each est.
302     £168-£196 each est.
305       £31-£40 each est.
307      £94-£115 each est.
311       £35-£41 each est.
313       £32-£39 each est.
318       £24-£32 each est.
48                      NaN
49                      NaN
48                      NaN
49                      NaN
3237      £42-£60 each est.
3279      £20-£28 each est.
3328     £55-£110 each est.
3330      £62-£79 each est.
3360      £30-£38 each est.
3378      £54-£92 each est.
3396      £60-£80 each est.
3435    £168-£196 each est.
3448      £31-£40 each est.
3454      £25-£38 each est.
3471     £94-£115 each est.
3475      £27-£39 each est.
3513      £35-£41 each est.
3536      £32-£39 each est.
3546      £39-£45 each est.
3562      £24-£32 each est.
48                      NaN
49                      NaN
24                      NaN
48                      NaN
49                      NaN
76        £62-£79 each est.
1193      £42-£60 ea

In [255]:
def clean_item_price(item_price: str):
    # Convert the input to a string and strip
    item_price = str(item_price).strip()
    
    # Remove 'each est'
    item_price = remove_each_est(item_price)

    # Remove currency symbol
    item_price = remove_currency(item_price)

    # Split on hyphen and keep first value
    item_price = get_first_number(item_price)

    # Account for 99p style numbers
    item_price = get_pence_value(item_price)   
   
    # Return the cleaned string as a string
    return item_price



def remove_currency(item_price: str):
    item_price = item_price.strip()
    return item_price.replace('£', '')

def remove_each_est(item_price: str):
    item_price = item_price.strip()
    return item_price.replace('each est.', '')

def get_first_number(item_price):
    item_price = item_price.strip()
    if '-' in item_price:
        return item_price.split('-')[0]
    else:
        return item_price

    
def get_pence_value(item_price):  
    item_price = item_price.strip()
    if item_price.endswith('p'):
        return '0.' + item_price.strip('p').strip()
    else:
        return item_price

In [264]:
df['item-price'] = df['item-price'].apply(clean_item_price)
df['item-price'] = df['item-price'].astype(float)
df.dropna(subset=['item-price'], inplace=True)

In [266]:
# confirming 0 missing values
df['item-price'].isna().sum()

0

In [251]:
missing_rows = df['item-price'].apply(clean_item_price).astype(float).isna()

In [254]:
df['item-price'][missing_rows]

48    NaN
49    NaN
48    NaN
49    NaN
48    NaN
49    NaN
24    NaN
48    NaN
49    NaN
48    NaN
49    NaN
Name: item-price, dtype: object